<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#(a)" data-toc-modified-id="(a)-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>(a)</a></span></li><li><span><a href="#(b)" data-toc-modified-id="(b)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>(b)</a></span></li><li><span><a href="#(c)" data-toc-modified-id="(c)-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>(c)</a></span></li><li><span><a href="#(d)" data-toc-modified-id="(d)-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>(d)</a></span></li></ul></li></ul></div>

Use the Gauss-Seidel method to approximate the fixed points in Exercise 7 to within $10^{−5}$ , using the
$l_{\infty}$ norm.

In [1]:
import numpy as np
from numpy import linalg
from abc import abstractmethod
import pandas as pd
import math

pd.options.display.float_format = '{:,.8f}'.format
np.set_printoptions(suppress=True, precision=8)

TOR = pow(10.0, -5)
MAX_ITR = 150

In [2]:
class FixedPointMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

## (a) 
$$G(x_1, x_2, x_3) = \left(\frac{\cos(x_2 x_3) + 0.5}{3},\frac{1}{25}\sqrt{x_1^2 + 0.3125} − 0.03, − \frac{1}{20}\exp^{−x_1 x_2 } −\frac{10\pi−3}{60}\right)$$

In [3]:
class FixedPointAcceleration(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = (math.cos(x[1] * x[2]) + 0.5) / 3.0
        sol[1] = math.sqrt(sol[0] * sol[0] + 0.3125) / 25.0 - 0.03
        sol[2] = -1.0 / 20.0 * math.exp(-sol[0] * sol[1]) - (10 * math.pi - 3) / 60.0
        return sol

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        while True:
            y = self.f(x)
            residual = linalg.norm(x - y, np.inf)
            x = y

            row = len(df)
            df.loc[row] = [ye for ye in y] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

In [4]:
x0 = np.array([0, 0, 0])
FixedPointAcceleration().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.00000000,0.00000000,0.00000000,nan,0.52359878
1,0.50000000,0.00000000,-0.52359878,0.52359878,0.00000000
2,0.50000000,0.00000000,-0.52359878,0.00000000,0.00000000


## (b) 
$$ G(x_1, x_2, x_3) = \left(\frac{13-x_2^2+4x_3}{15}, \frac{11+x_3-x_1^2}{10}, \frac{22+x_2^3}{25}\right)$$

In [5]:
class FixedPointAcceleration(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = (13 - pow(x[1], 2) + 4 * x[2]) /15.0
        sol[1] = (11 + x[2] - pow(sol[0], 2)) /10.0 
        sol[2] = (22 + pow(sol[1], 3)) / 25.0
        return sol

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        while True:
            y = self.f(x)
            residual = linalg.norm(x - y, np.inf)
            x = y

            row = len(df)
            df.loc[row] = [ye for ye in y] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

In [6]:
x0 = np.array([0.75, 0.75, 0.75])
FixedPointAcceleration().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.75000000,0.75000000,0.75000000,nan,0.33570652
1,1.02916667,1.06908160,0.92887565,0.31908160,0.01662492
2,1.03817114,1.08510763,0.93110677,0.01602604,0.00177059
3,1.03646457,1.08568480,0.93118837,0.00170658,0.00006401
4,1.03640280,1.08570576,0.93119133,0.00006177,0.00000224
5,1.03640055,1.08570652,0.93119144,0.00000224,0.00000000


## (c) 
$$ G(x_1, x_2, x_3) = \left( 1-\cos(x_1 x_2 x_3), 1-(1-x_1)^{\frac{1}{4}} - 0.05x_3^2+0.15x_3, x_1^2+0.1x_2^2-0.01x_2+1 \right) $$

In [7]:
class FixedPointAcceleration(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 1 - math.cos(x[0] * x[1] * x[2])
        sol[1] = 1 - pow((1 - sol[0]), 1 / 4) - 0.05 * pow(x[2], 2) + 0.15 * x[2]
        sol[2] = pow(sol[0], 2) + 0.1 * pow(sol[1], 2) - 0.01 * sol[1] + 1
        return sol

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        while True:
            y = self.f(x)
            residual = linalg.norm(x - y, np.inf)
            x = y

            row = len(df)
            df.loc[row] = [ye for ye in y] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

In [8]:
x0 = np.array([0, 0, 0])
FixedPointAcceleration().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.00000000,0.00000000,0.00000000,nan,1.00000000
1,0.00000000,0.00000000,1.00000000,1.00000000,0.10000000
2,0.00000000,0.10000000,1.00000000,0.10000000,0.00000000
3,0.00000000,0.10000000,1.00000000,0.00000000,0.00000000


## (d) 
$$ G(x_1, x_2, x_3) = \left( \frac{1}{3}\cos(x_2 x_3)+\frac{1}{6}, -\frac{1}{9}\sqrt{x_1^2+\sin x_3 +1.06}-0.1,− \frac{1}{20}\exp^{−x_1 x_2 } −\frac{10\pi−3}{60} \right) $$

In [9]:
class FixedPointAcceleration(FixedPointMethod):

    def __init__(self):
        super(FixedPointMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = math.cos(x[1] * x[2]) / 3.0 + 1.0 / 6.0
        sol[1] = - 1 / 9 * math.sqrt(pow(sol[0], 2) + math.sin(x[2]) + 1.06) - 0.1
        sol[2] = -1.0 / 20.0 * math.exp(-sol[0] * sol[1]) - (10 * math.pi - 3) / 60.0
        return sol

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        while True:
            y = self.f(x)
            residual = linalg.norm(x - y, np.inf)
            x = y

            row = len(df)
            df.loc[row] = [ye for ye in y] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        return df

In [10]:
x0 = np.array([0, 0, 0])
FixedPointAcceleration().run(x0)

,x1,x2,x3,residual,actual-residual
0,0.00000000,0.00000000,0.00000000,nan,0.52882598
1,0.50000000,-0.22717248,-0.52961320,0.52961320,0.02756659
2,0.49759035,-0.19952955,-0.52881777,0.02764293,0.00055432
3,0.49814616,-0.19960643,-0.52882601,0.00055581,0.00000149
4,0.49814467,-0.19960589,-0.52882598,0.00000149,0.00000000
